In [13]:
import numpy as np
from scipy.stats import chi2
from tqdm.notebook import tqdm

In [2]:
def make_rectangle(dot, i):
    return ((dot[0], dot[1]), (dot[0], dot[1] + i), (dot[0] + i, dot[1] + i), (dot[0] + i, dot[1]))

In [3]:
# Функция отражения прямоугольника симметрично относительно нуля
def reverse_rectangle(rectangle):
    
    dot1 = rectangle[2]
    dot2 = rectangle[3]
    dot3 = rectangle[0]
    dot4 = rectangle[1]
    
    return ((-dot1[0], -dot1[1]), (-dot2[0], -dot2[1]), (-dot3[0], -dot3[1]), (-dot4[0], -dot4[1]))

In [5]:
def is_dot_in_rectangle(dot, rectangle):
    
    return dot[0] >= rectangle[0][0] \
            and dot[0] <= rectangle[2][0] \
            and dot[1] >= rectangle[0][1] \
            and dot[1] <= rectangle[2][1]

# Разобьем правую полуплоскость на квадраты

In [6]:
dots = []
for i in range(0, 11, 2):
    for j in range(-10, 11, 2):
        dots.append((i, j))

In [7]:
rectangles = [make_rectangle(dot, 1) for dot in dots]

In [9]:
rectangles[5]

((0, 0), (0, 1), (1, 1), (1, 0))

In [10]:
is_dot_in_rectangle((0.5, 0.5), rectangles[5]), is_dot_in_rectangle((1.5, 0.5), rectangles[5])

(True, False)

## Функция подсчета статистики Т

$$T = \sum\limits_{i=1}^{m}\frac{(\nu_{i}^{+} - \nu_{i}^{-})^{2}}{\nu_{i}^{+} + \nu_{i}^{-}}$$

In [11]:
def t_statistics(vecs: np.array):
    
    rectangles_dict = {}
    reversed_rectangles = [reverse_rectangle(rectangle) for rectangle in rectangles]
    
    for rectangle in rectangles + reversed_rectangles:
        rectangles_dict[rectangle] = 0
    

    for dot in vecs:
        for rectangle in rectangles + reversed_rectangles:
            
            if is_dot_in_rectangle(dot, rectangle):
                rectangles_dict[rectangle] += 1
                
    to_count_rectangles = []
    T = 0
    for rectangle in rectangles:
        
        if rectangles_dict[rectangle] > 0 or rectangles_dict[reverse_rectangle(rectangle)] > 0:
            to_count_rectangles.append(rectangle)
            
            T += (
                (rectangles_dict[rectangle] - rectangles_dict[reverse_rectangle(rectangle)])**2 /
                (rectangles_dict[rectangle] + rectangles_dict[reverse_rectangle(rectangle)])
            )
                            
    degree_freedom = len(to_count_rectangles)
    return T, degree_freedom

### Сгенерируем 1000 раз выборку из симметричного относительно нуля двумерного нормального распределения. То есть нулевая гипотеза о симметрии распределения верна.
$$\mathcal{N} (\begin{pmatrix}
0\\
0
\end{pmatrix}, \begin{pmatrix}
9 & 2\\
2 & 16
\end{pmatrix})$$
#### В случае отклонения нулевой гипотезы о симметрии, будем записывать ответ 1, иначе - 0.

In [28]:
answers = []
for _ in tqdm(range(1000)):
    
    mean = np.array([0, 0])
    cov = np.array([[9, 2], [2, 16]])
    vecs = np.random.multivariate_normal(mean, cov, 1000)
    
    T, df = t_statistics(vecs)
    
    if T > chi2.ppf(0.95, df=df):
        answers.append(1)
    else:
        answers.append(0)
np.mean(answers)

  0%|          | 0/1000 [00:00<?, ?it/s]

0.015

### Как видим, мы отклонили нулевую гипотезу лишь в 1.5% случаев. Это и есть ошибка первого рода.

_____

### Сгенерируем теперь 1000 раз выборку из несимметричного двумерного нормального распределения с той же ковариационной матрицей, но с ненулувым вектором средних. То есть нулевая гипотеза о симметрии распределения не верна.
$$\mathcal{N} (\begin{pmatrix}
-1\\
1
\end{pmatrix}, \begin{pmatrix}
9 & 2\\
2 & 16
\end{pmatrix})$$

In [27]:
answers = []
for _ in tqdm(range(1000)):
    
    mean = np.array([-1, 1])
    cov = np.array([[9, 2], [2, 16]])
    vecs = np.random.multivariate_normal(mean, cov, 1000)
    
    T, df = t_statistics(vecs)
    
    if T > chi2.ppf(0.95, df=df):
        answers.append(1)
    else:
        answers.append(0)

np.mean(answers)

  0%|          | 0/1000 [00:00<?, ?it/s]

0.957

### Как видим, мы отклонили неверную нулевую гипотезу лишь в 95.7% случаев. То есть ошибка второго равна 4.3%.

# Таким образом мы подтвердили валидность теста. Ошибки первого и второго рода составили меньше 5%.